<a href="https://colab.research.google.com/github/venkatanadikatla/pytorch/blob/main/Hierarchical_Normal_Model_%26_Gibbs_Sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Overview:**
The dataset consists of annual death counts from five causes for the age group 25–34 over five years (2018–2022). These causes are:

Accidents, Homicide, Suicide, Heart Disease, Malignant Neoplasms
The goal is to model these death counts using three approaches:


*   Separate Gaussian model

*   Pooled Gaussian model
*   Hierarchical Gaussian model

**Why Gibbs Sampling:**
Gibbs sampling is a special case of the Metropolis-Hastings algorithm used for Bayesian inference, particularly when the full conditional distributions of the variables in a model can be easily sampled.

**a. Separate Model:** In a separate model, we treat each group or data source independently. When using Gibbs sampling here, each group's parameters are sampled independently from their respective conditional distributions.



*   **Specific to the model:**  Sample the parameters for each cause of death independently. Each death cause has its own parameters, which could lead to high variability in small samples.


**b. Pooled Model:** In a pooled model, the data from different groups are combined (or "pooled") and treated as if they came from a single distribution. Gibbs sampling in this model would sample the parameters from a single conditional distribution across all data. This reduces variance but may miss group-specific effects, assuming all groups are homogenous.

*   **Specific to the model:**  Pooled Model (Gibbs): Sample from a common parameter for all causes, assuming homogeneity.

**c. Hierarchical Model** (or Partially Pooled Model): In hierarchical models, group-level parameters (for individual groups) are treated as random variables drawn from a higher-level (hyperparameter) distribution. Gibbs sampling here involves sampling both the group-level parameters and the hyperparameters in a conditional step-by-step process. This approach balances between separate and pooled models, allowing group-specific variability while sharing information across groups.

*   **Specific to the model:** Sample group-specific parameters and hyperparameters, allowing for shared information while maintaining some independence between causes.




**General Setup:**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, gamma

# Death data for the five causes from 2018 to 2022
deaths = np.array([[33058, 34452, 31315, 24516, 24614],   # Accidents
                   [6712, 7571, 31315, 5341, 5234],       # Homicide
                   [8663, 8862, 8454, 8059, 8020],        # Suicide
                   [3789, 4155, 3984, 3495, 3561],        # Heart Disease
                   [3641, 3615, 3573, 3577, 3684]])       # Malignant Neoplasms

n_years, n_causes = deaths.shape

**1. Separate Gaussian Model:** Each cause has its own mean and shared variance

In [ ]:
separate_gaussian <- function(deaths, n_iter = 5000, burn_in = 1000) {
  n_years <- nrow(deaths)
  n_causes <- ncol(deaths)

  # Prior hyperparameters
  mu_prior_mean <- 0
  mu_prior_variance <- 100^2
  alpha <- 0.001
  beta <- 0.001

  # Initialize parameters
  mu <- apply(deaths, 2, mean)
  sigma2 <- 1000

  # Storage for samples
  mu_samples <- matrix(NA, nrow = n_iter, ncol = n_causes)
  sigma2_samples <- numeric(n_iter)

  for (iter in 1:n_iter) {
    # Update mu for each cause
    for (c in 1:n_causes) {
      y_c <- deaths[, c]
      n <- length(y_c)
      mu_post_mean <- (sum(y_c) / sigma2 + mu_prior_mean / mu_prior_variance) /
                      (n / sigma2 + 1 / mu_prior_variance)
      mu_post_var <- 1 / (n / sigma2 + 1 / mu_prior_variance)
      mu[c] <- rnorm(1, mean = mu_post_mean, sd = sqrt(mu_post_var))
    }

    # Update sigma^2
    residuals <- deaths - matrix(rep(mu, each = n_years), n_years, n_causes)
    ss <- sum(residuals^2)
    sigma2_post_shape <- alpha + 0.5 * n_years * n_causes
    sigma2_post_rate <- beta + 0.5 * ss
    sigma2 <- 1 / rgamma(1, shape = sigma2_post_shape, rate = sigma2_post_rate)

    # Store samples
    mu_samples[iter, ] <- mu
    sigma2_samples[iter] <- sigma2
  }

  # Remove burn-in period
  mu_samples <- mu_samples[(burn_in+1):n_iter, ]
  sigma2_samples <- sigma2_samples[(burn_in+1):n_iter]

  list(mu_samples = mu_samples, sigma2_samples = sigma2_samples)
}

separate_results <- separate_gaussian(deaths)


In [ ]:
posterior_means_separate <- apply(separate_results$mu_samples, 2, mean)
posterior_means_separate


[1] 1.052668 3.383939 4.130712 1.521936 4.188088

In [ ]:
mu_6_separate <- rnorm(1, mean = mean(posterior_means_separate),
                       sd = sqrt(mean(separate_results$sigma2_samples)))

# Predictive distribution for the 6th cause
y_pred_separate <- rnorm(1000, mean = mu_6_separate,
                         sd = sqrt(mean(separate_results$sigma2_samples)))


[1]  23455.338404   6818.028542   6766.497714 -10384.629216 -15509.816393
   [6]   6490.710356  18499.106698 -15326.300293  -3049.358836 -13017.591160
  [11]  -6417.520812  10089.915851  12503.530239  -4582.897611  12287.891427
  [16]  -5388.018609  -7671.817496 -16028.774461 -12819.215462  23612.467176
  [21]  27886.737984   2400.738821   3132.889179 -13829.393159  23240.990881
  [26]  13762.773745    726.455213 -30574.257948 -18439.863219   3159.166038
  [31] -30587.462969 -23341.440749 -58449.551017   -966.320523   7772.205403
  [36]  19285.054941   5448.556866 -15072.717956  -1762.307863  21775.785167
  [41]  -6678.155448  40663.487394  -9095.006694  24704.729201  19033.042581
  [46]  -5133.424398  -5856.381505   8063.470746  34316.393189 -15971.396004
  [51] -13972.899363   6963.212010 -12939.444143 -12319.238862  11862.711520
  [56]  10243.578585   5793.125754    -32.428190 -40377.380953 -17384.411359
  [61]  12454.341428   4356.280706   4343.119142   3631.938095  -6499.161650
  [66]  16550.114308   -257.775322  20294.165716 -15282.943052  25682.023840
  [71]  26536.280742 -14784.923825  -6245.859499 -27295.360899 -24968.413003
  [76]  -7393.566207  14655.491882   2773.440794 -35803.470638  17377.638932
  [81] -10180.960838  10013.442222 -17654.873489  -5324.049301 -10936.513126
  [86]  16328.096411 -28856.112394  -8164.442759  -3717.638097    705.102515
  [91]  -6442.057529  12315.679204   3472.970308  25660.274226  43117.175403
  [96]   1204.591902  12962.661689 -24387.234166   2817.330629  12968.155979
 [101]  -7704.880955  -8533.542226 -13958.818786 -12494.949179  -3396.098171
 [106]  -8377.984466  -1915.485546  23424.041722  26833.391899  -6706.115367
 [111] -30379.499154  12844.489471  12717.873635   4190.982091  -6464.943539
 [116] -13776.667042   4327.673344 -24389.454851   1032.941531   -474.958807
 [121] -10165.241786   5553.947009  -3364.113360 -18723.162680   -493.670256
 [126]   6919.504965  16759.945210  28680.143400   3843.287000    894.178710
 [131]  22543.135384   6704.276173   9523.621193 -20581.968921   8353.195178
 [136]  27808.881803   -933.239196   3342.411551   -264.454630 -20997.376997
 [141] -19332.509003  -1749.725789   9553.991676  -7516.311638 -26706.295364
 [146]  -9988.141894   1340.228729   5888.169153 -12779.314012  -1341.902250
 [151] -10039.292082 -12465.478659  31327.718687  -2071.011323  -8391.260464
 [156]  22228.914805  -9347.460727   4503.800437   3888.365017   5770.368819
 [161] -17671.635265   -906.280895   3406.517166   1487.562313 -25680.618114
 [166]  14446.856874 -26127.521236  33751.343109   4194.409671 -13857.209391
 [171] -19431.768168  -2401.861832 -12149.740237    280.167731  36496.566197
 [176]  -1101.405957  -8055.746197 -31702.526678 -18067.810096     -7.297602
 [181]  -9428.253156   5800.169189    361.653309 -15602.239879 -16215.131193
 [186]  -3276.478931 -17742.518782  19238.912322 -39047.368871  15711.554458
 [191]  23839.298014  10461.860972 -23095.930747 -11604.584427   8151.966277
 [196]  17481.359263   1374.349872  -3071.025289  28283.522129  -5918.504643
 [201] -17831.352181 -15992.756413  23052.295705  13710.243635 -14622.539636
 [206]  18001.220208 -23286.143869 -21321.972522  -7070.839051  -6614.544950
 [211]  -5196.194248 -12904.898277   7737.254374   5134.676829   9161.774131
 [216]  -4896.067983 -12908.659000  42217.568298  -7192.589639 -34776.294790
 [221]   2404.752261 -27138.097476  -5335.308333  10443.902697 -32988.129315
 [226] -13922.230749 -12026.748448   8129.435421 -11891.328119  11461.538846
 [231] -13665.370001   9873.195679  39734.896730 -30190.811821  -2677.104996
 [236]   9833.923833   2246.716697 -11301.282546   4265.710824    297.832696
 [241]  17738.712865 -26475.620389  -7166.021377  18829.032115 -11353.647125
 [246]  -7698.697194  14671.780221  20361.633399  47781.601447  -8844.897402
 [251]  17568.527126   1372.571093  13047.609217  -3613.233866   9564.433070
 [256]  13113.968790  -4143.927301 -25693.290022  10035.717548  -1509.512706